In [2]:
import requests

In [154]:
BACKEND_HOST_IP="10.74.4.25"
KONG_URL="http://localhost:8001"
BACKEND_URI="http://10.74.4.25:5000/api"
FRONTEND_URI="http://10.74.4.25:4000/"
AUTH_URI="http://10.65.98.41:8080/"

KONG_CLIENT_ID="kong"
KONG_CLIENT_SECRET="7af12c9d-47b1-4a53-91ac-9acf49b704ef"
REALM_NAME="enki"
SGE_API_KEY = "e86366d6-8ae7-42e5-99b1-cdd04cc9b5b7"

In [155]:
KEYCLOAK_URL = "https://auth.enki.nexsis112.fr"

In [156]:
introspection_url = f'{KEYCLOAK_URL}/auth/realms/{REALM_NAME}/protocol/openid-connect/token/introspect'
discovery_url = f'{KEYCLOAK_URL}/auth/realms/{REALM_NAME}/.well-known/openid-configuration'


In [157]:
discovery_url

'https://auth.enki.nexsis112.fr/auth/realms/enki/.well-known/openid-configuration'

In [158]:
services = [
    {
        'name': "enki_api",
        'url': BACKEND_URI,

        "routes":[
            {
                "path": '/api',
                'auth':'oidc'
            }, 
            {
                'path':'/echanges', 
                'auth':'api_key'
            }
        ]
    },
        {
        'name': "enki_front",
        'url': FRONTEND_URI,
        'routes':[
             {         
                'path': '/',
                "auth":"oidc"
             }   
        ]
    }
]

In [159]:
response = requests.get(f'{KONG_URL}/routes')
for _id in [e["id"] for e in response.json()["data"]]:
    requests.delete(f'{KONG_URL}/routes/{_id}')
response = requests.get(f'{KONG_URL}/services')
for _id in [e["id"] for e in response.json()["data"]]:
    requests.delete(f'{KONG_URL}/services/{_id}')

In [160]:
for service in services : 
    response = requests.post(f'{KONG_URL}/services', data={
        'name': service["name"],
        'url': service["url"]
    })
    print(response.json())
    created_service_id = response.json()["id"]
    for route in service["routes"]:
        response = requests.post(f'{KONG_URL}/services/{service["name"]}/routes', data={
            'service.id': created_service_id,
            'paths[]':route["path"],
        })
        route_id = response.json()["id"]
        auth_type=route.get("auth")
        if auth_type == "api_key":
            response = requests.post(f'{KONG_URL}/routes/{route_id}/plugins', data={
                'name': "key-auth",
                'config.key_names': "x-api-key",
            })
            response = requests.post(f'{KONG_URL}/consumers', data={
                'username': "sge-hub",
            })
            print(response.status_code)  
            print(response.json())
            response = requests.post(f'{KONG_URL}/consumers/sge-hub/key-auth', data={
                    'key': apikey,
                })
            print(response.status_code)
        elif auth_type == "oidc":
            data = {
                    'name': 'oidc',
                    'config.client_id': f'{KONG_CLIENT_ID}',
                    'config.client_secret': f'{KONG_CLIENT_SECRET}',
                    'config.realm': f'{REALM_NAME}',
                    'config.bearer_only': 'false',
                    'config.introspection_endpoint': introspection_url,
                    'config.discovery': discovery_url
            }

            response = requests.post(f'{KONG_URL}/services/{created_service_id}/plugins', data=data)
        else:
            pass

{'host': '10.74.4.25', 'id': '0af7898e-3b56-4fa7-9edd-38b562f3390c', 'protocol': 'http', 'read_timeout': 60000, 'tls_verify_depth': None, 'port': 5000, 'updated_at': 1619079345, 'ca_certificates': None, 'created_at': 1619079345, 'connect_timeout': 60000, 'write_timeout': 60000, 'name': 'enki_api', 'retries': 5, 'path': '/api', 'tls_verify': None, 'tags': None, 'client_certificate': None}
409
{'message': 'UNIQUE violation detected on \'{username="sge-hub"}\'', 'name': 'unique constraint violation', 'fields': {'username': 'sge-hub'}, 'code': 5}
409
{'host': '10.74.4.25', 'id': '05f1de98-307b-42ce-b812-207fb4e684b9', 'protocol': 'http', 'read_timeout': 60000, 'tls_verify_depth': None, 'port': 4000, 'updated_at': 1619079345, 'ca_certificates': None, 'created_at': 1619079345, 'connect_timeout': 60000, 'write_timeout': 60000, 'name': 'enki_front', 'retries': 5, 'path': '/', 'tls_verify': None, 'tags': None, 'client_certificate': None}


In [130]:
response.json()

{'config': {'bearer_only': 'false',
  'client_id': 'kong',
  'client_secret': '7af12c9d-47b1-4a53-91ac-9acf49b704ef',
  'discovery': 'https://auth.enki.nexsis112.fr/auth/realms/enki/.well-known/openid-configuration',
  'filters': None,
  'introspection_endpoint': 'https://auth.enki.nexsis112.fr/auth/realms/enki/protocol/openid-connect/token/introspect',
  'introspection_endpoint_auth_method': None,
  'logout_path': '/logout',
  'realm': 'enki',
  'recovery_page_path': None,
  'redirect_after_logout_uri': '/',
  'redirect_uri_path': None,
  'response_type': 'code',
  'scope': 'openid',
  'session_secret': None,
  'ssl_verify': 'no',
  'token_endpoint_auth_method': 'client_secret_post'},
 'consumer': None,
 'created_at': 1619018597,
 'enabled': True,
 'id': '983c114e-11e4-4733-b59c-4952732fb21b',
 'name': 'oidc',
 'protocols': ['grpc', 'grpcs', 'http', 'https'],
 'route': None,
 'service': {'id': 'd5ed9277-26b6-48ea-bc8e-93d96ffb2c58'},
 'tags': None}